Task 1

In [112]:
import numpy as np
import pandas as pd

arr = np.random.rand(10, 5)
print(arr)

df = pd.DataFrame(arr)
res = df[df > 0.3].mean(axis = 1) 
res

[[0.82544199 0.86116806 0.50556091 0.8506465  0.31275744]
 [0.29155679 0.74044802 0.64449473 0.26238747 0.30193321]
 [0.80041699 0.43601168 0.26832861 0.38370572 0.92307753]
 [0.77737399 0.82596634 0.73975309 0.06656691 0.24187406]
 [0.43296664 0.51877584 0.32783555 0.68265247 0.58080971]
 [0.80932761 0.45063843 0.84512128 0.58320129 0.43484768]
 [0.55611759 0.25244121 0.09771773 0.57557205 0.1706126 ]
 [0.19247076 0.09839424 0.26517194 0.62573399 0.00566652]
 [0.91563954 0.06506386 0.28346615 0.51047873 0.53040281]
 [0.02029711 0.97999884 0.64609796 0.0514741  0.86705092]]


0    0.671115
1    0.562292
2    0.635803
3    0.781031
4    0.508608
5    0.624627
6    0.565845
7    0.625734
8    0.652174
9    0.831049
dtype: float64

Task 2

In [113]:
df = pd.read_csv('./wells_info.csv')

period = lambda col: pd.to_datetime(df[col]).dt.to_period('M').astype(np.int64)
res = (period('CompletionDate') - period('PermitDate'))

res

0      7
1      7
2     12
3     18
4      3
5      8
6      6
7     16
8      8
9     12
10     9
11     3
12     7
13     6
14     8
15    12
16    20
17     9
18    12
19     5
20    11
21    10
22     5
23    12
24     7
25     8
26     6
27     9
28     7
29     7
30    10
31     7
32    11
33     6
34     8
35     2
36     3
37     9
38     4
39     4
40     4
41     5
42    16
43     5
44     6
45    14
46     8
47     5
48     9
49     6
dtype: int64

Task 3

In [114]:
df = pd.read_csv('./wells_info_na.csv')

lst = df.to_numpy(copy = True, na_value = np.nan).ravel().tolist()
arr, freq = np.unique(lst, return_counts = True)

lst.sort(key = lambda num: freq[np.where(arr == str(num))][0], reverse = True)
most_freq = list(filter(lambda x: str(x) != 'nan', lst))[0]

df = df.apply(lambda x: pd.to_datetime(x, errors = 'ignore') if x.dtype == 'object' else x)
df = df.apply(lambda x: x.fillna(x.quantile(0.5, interpolation="midpoint")) if x.dtype == 'datetime64[ns]' else x)
df = df.fillna(df.median(numeric_only = True)).fillna(most_freq)

df

,API,CompletionDate,FirstProductionDate,formation,BasinName,StateName,CountyName,LatWGS84,LonWGS84,PROP_PER_FOOT
0,5123402600000,2015-09-13,2015-09-01,CODELL,DENVER BASIN,COLORADO,WELD,40.353967,-104.944679,926.5010
1,5123409020000,2015-09-07,2015-09-01,NIOBRARA,WELD,COLORADO,WELD,40.482361,-104.468261,697.0967
2,5123410480000,2015-04-07,2015-04-01,NIOBRARA,DENVER BASIN,WELD,WELD,40.482361,-103.561306,896.9146
3,30015414470000,2014-01-07,2013-12-01,BONE SPRING,PERMIAN BASIN,NEW MEXICO,EDDY,40.482361,-103.561306,648.1232
4,30025425390000,2015-10-10,2015-08-01,WOLFCAMP UPPER,PERMIAN BASIN,NEW MEXICO,LEA,32.080410,-103.530040,1657.9800
5,33053048870100,2014-11-05,2014-11-01,BAKKEN,WELD,NORTH DAKOTA,MCKENZIE,47.995019,-103.561306,984.4741
6,33053051530000,2014-04-27,2014-04-01,BAKKEN,WILLISTON BASIN,NORTH DAKOTA,MCKENZIE,47.992928,-103.020490,341.2530


Task 4

In [115]:
df_wells = pd.read_csv('./wells_info.csv')
df_prod = pd.read_csv('./production.csv')

df = df_prod.groupby('API').sum(numeric_only = True)
df['FullProd'] = df['Water'] + df['Gas'] + df['Liquid']

df1 = df_prod[['API','Liquid', 'Water', 'Gas']].set_index('API').groupby('API').apply(lambda x: x[:12].sum(numeric_only = True))
df1['ProdFor12M'] = df1['Water'] + df1['Gas'] + df1['Liquid']

df_wells = df_wells.merge(df['FullProd'], on = 'API').merge(df1['ProdFor12M'], on = 'API')

df_wells.head()

,API,PermitDate,SpudDate,CompletionDate,FirstProductionDate,operatorNameIHS,formation,BasinName,StateName,CountyName,LatWGS84,LonWGS84,BottomHoleLatitude,BottomHoleLongitude,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT,FullProd,ProdFor12M
0,5005072170100,2014-05-06,2014-05-06,2014-12-02,2014-11-01 00:00:00.000,BURLINGTON RESOURCES O&G CO LP,NIOBRARA,DENVER BASIN,COLORADO,ARAPAHOE,39.684606,-104.642128,39.68445,-104.60557,9005.0,994.6866,591.8004,629543,393066
1,5123377130000,2013-07-21,2013-10-06,2014-02-26,2014-02-01 00:00:00.000,SRC ENERGY INCORPORATED,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.509320,-104.780980,40.49692,-104.77859,4195.0,991.5857,628.6321,176949,110847
2,5123379280000,2013-09-08,2013-09-25,2014-09-07,2013-11-01 00:00:00.000,BONANZA CREEK ENERGY OPERATING CO LLC,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.335390,-104.363000,40.34780,-104.36863,4273.0,1000.2760,564.4841,239200,152298
3,5123379400000,2013-09-11,2014-12-05,2015-03-31,2015-03-01 00:00:00.000,NOBLE ENERGY INCORPORATED,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.152220,-104.530780,40.17445,-104.52932,7078.0,973.4437,824.0020,464291,317982
4,5123385820100,2014-01-07,2014-01-13,2014-04-23,2014-04-01 00:00:00.000,GREAT WESTERN OPERATING COMPANY LLC,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.508303,-104.868180,40.49558,-104.86757,3211.0,783.5919,603.1414,104618,74479


Laboratory 5.1

In [116]:
titanic = pd.read_csv('./titanic_with_labels.csv', sep = ' ', index_col = 0)

titanic.sex = titanic.sex.apply(str.lower)
titanic = titanic[(titanic.sex == 'м') | (titanic.sex == 'ж')]
titanic.sex = titanic.sex.replace({'м' : 0, 'ж' : 1})

titanic.row_number = titanic.row_number.fillna(titanic.row_number.max(skipna = True))

titanic.liters_drunk = titanic.liters_drunk.apply(lambda x: np.nan if x < 0 or x > 10 else x)
titanic.liters_drunk = titanic.liters_drunk.fillna(titanic.liters_drunk.mean())

titanic.head(20)

,sex,row_number,liters_drunk,age,drink,check_number,label
0,1,79.0,2.000000,66,Cola,3116,0
1,0,23.0,0.000000,10,Fanta,8118,0
3,1,52.0,5.000000,67,Beerbeer,7640,1
4,1,57.0,2.000000,27,Beerbeer,8764,0
5,0,15.0,1.000000,44,Bugbeer,6235,0
6,1,49.0,0.000000,11,Cola,185,1
7,0,33.0,5.000000,64,Fanta,2970,1
8,1,5.0,0.000000,39,Water,6494,0
10,1,78.0,2.469941,51,Water,3592,0
11,0,32.0,2.000000,35,Strong beer,2021,0


Laboratory 5.2

In [117]:
sessions = pd.read_csv('./cinema_sessions.csv',  sep = ' ', index_col = 0, parse_dates = ['session_start'])

titanic['age_children'] = np.where(titanic.age < 18, titanic.age, None)
titanic['age_adults'] = np.where((titanic.age >= 18) & (titanic.age < 50), titanic.age, None)
titanic['age_elderly'] = np.where(titanic.age >= 50, titanic.age, None)
titanic = titanic.drop(columns = 'age')

titanic.drink = titanic.drink.apply(lambda x: 1 if x.find('beer') != -1 else 0)

sessions['morning_session'] = pd.to_datetime(np.where(sessions.session_start.dt.hour < 12, sessions.session_start, None))
sessions['day_session'] = pd.to_datetime(np.where((sessions.session_start.dt.hour >= 12) & (sessions.session_start.dt.hour < 18), sessions.session_start, None))
sessions['evening_session'] = pd.to_datetime(np.where(sessions.session_start.dt.hour >= 18, sessions.session_start, None))
sessions = sessions.drop(columns = 'session_start')

titanic = pd.merge(titanic, sessions, on = 'check_number')

titanic

,sex,row_number,liters_drunk,drink,check_number,label,age_children,age_adults,age_elderly,morning_session,day_session,evening_session
0,1,79.0,2.0,0,3116,0,None,None,66,NaT,NaT,2022-11-26 21:00:00
1,0,23.0,0.0,0,8118,0,10,None,None,NaT,2022-11-26 15:30:00,NaT
2,1,52.0,5.0,1,7640,1,None,None,67,NaT,NaT,2022-11-26 23:42:00
3,1,57.0,2.0,1,8764,0,None,27,None,NaT,2022-11-26 15:30:00,NaT
4,0,15.0,1.0,1,6235,0,None,44,None,2022-11-26 08:42:00,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
4994,1,61.0,3.0,1,774,1,None,None,50,NaT,NaT,2022-11-26 23:42:00
4995,1,100.0,3.0,0,1022,1,15,None,None,NaT,2022-11-26 15:00:00,NaT
4996,0,15.0,3.0,1,6510,0,None,None,62,NaT,NaT,2022-11-26 22:15:00
4997,1,11.0,3.0,1,9510,1,None,None,52,NaT,NaT,2022-11-26 22:15:00
